# Import Packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import io
import datetime

## Define Function

In [2]:
# Create function to do all combos
def get_matchup(df,
                pollster_rating,
                candidate_x, candidate_y,
                cand_x_party, cand_y_party,
                level = "national",
                state = None):
    """
    Takes in a Pandas DataFrame of polls from FiveThirtyEight
    and candidate names and returns a combined DataFrame of
    the Candidate-1 vs. Candidate-2 poll results.
    
    Parameters:
    -----------
    df : pandas dataframe, polls from FiveThirtyEight
    pollster_rating : pandas dataframe, pollster organization weights
    candidate_x : str, Name of candidate
    candidate_y : str, Name of candidate
    cand_x_party : str, Party of candidate
    cand_y_party : str, Party of candidate
    level : str, "national", "state", or "all" state level polls
    state : str, default None, if "state" level is specified, which state to return
    """
    
    # Get relevant columns from the DataFrame
    cols = ["question_id",
            "poll_id",
            "stage",
            "office_type",
            "pollster",
            "state",
            "end_date",
            "candidate_name",
            "candidate_party",
            "sample_size",
            "pct"]
    df = df[cols]
    
    
    # Combine polling data with pollster ratings
    cols = ["Pollster",
            "Predictive Plus-Minus",
            "538 Grade"]
    df = df.merge(pollster_rating[cols],
                         how = "left",
                         left_on = "pollster",
                         right_on = "Pollster")
    # Find and keep only polls with Pollster Ratings
    df = df.loc[df["pollster"] == df["Pollster"], ]
    
    
    
    # Get polls where there is no specified state (i.e. National Polls)
    if level == "national": 
        df = df.loc[df["state"].isna()]
    # Get polls for a specific state
    elif level == "state":
        df = df.loc[df["state"] == state]
    elif level == "all":
        df = df.loc[df["state"].notna()]
        
    # Create separate DataFrames for each candidate
    dict_of_candidate_dfs = {}
    for candidate in df['candidate_name'].unique():
        c_df = df.loc[df['candidate_name'] == candidate, :]
        dict_of_candidate_dfs[candidate] = c_df
    
    # Get each DataFrame for each candidate, if they exist
    if candidate_x in df["candidate_name"].unique():
        cand_1 = dict_of_candidate_dfs[candidate_x]
    else:
        print("No polls have been collected for the specified race.")
        return
    
    if candidate_y in df["candidate_name"].unique():
        cand_2 = dict_of_candidate_dfs[candidate_y]
    else:
        print("No polls have been collected for the specified race.")
        return
    
    # Combine DataFrames
    pair = pd.merge(cand_1,
                    cand_2[["question_id", "candidate_name", "candidate_party", "pct"]],
                    on = "question_id",
                    suffixes = (f"_{cand_x_party}", f"_{cand_y_party}")).drop(["Pollster"], axis = 1)
    
    # Create column for votes for each candidate
    pair[f"votes_{cand_x_party}"] = (pair["sample_size"] * (pair[f"pct_{cand_x_party}"] / 100)).astype(int)
    pair[f"votes_{cand_y_party}"] = (pair["sample_size"] * (pair[f"pct_{cand_y_party}"] / 100)).astype(int)
    
    pair["total_decided"] = pair[f"votes_{cand_x_party}"] + pair[f"votes_{cand_y_party}"]
    
    # Convert End Date column to Datetime
    pair["end_date"] = pd.to_datetime(pair["end_date"])
    
    return pair

# Get Polling Data

In [3]:
# Get recent Polling data
polls = pd.read_csv("../datasets/president_polls.csv")

In [4]:
# Create Pollster Ratings DataFrame from 538
url = "https://raw.githubusercontent.com/fivethirtyeight/data/master/pollster-ratings/pollster-ratings.csv"
res = requests.get(url).content
pollster_rating = pd.read_csv(io.StringIO(res.decode('utf-8')))

In [5]:
# Get example matchup polls
polls = get_matchup(polls, pollster_rating, "Donald Trump", "Elizabeth Warren", "rep", "dem", level = "all")

## Weight for Recency

In [6]:
# Create columns with the month and year of the polls
month_array = []
for i in range(len(polls["end_date"])):
    month_array.append(int(str(polls["end_date"][i])[5:7]))
    
year_array = []
for i in range(len(polls["end_date"])):
    year_array.append(int(str(polls["end_date"][i])[:4]))

polls["month"] = pd.Series(month_array)
polls["year"] = pd.Series(year_array)

In [7]:
# Get the month and year of the most recent poll
latest_poll = [polls["month"][0], polls["year"][0]]

In [8]:
# Convert months and year into distance in months from most recent poll
month_diff = []
for i in range(polls.shape[0]):
    months_back = latest_poll[0] - polls["month"][i]
    years_back = latest_poll[1] - polls["year"][i]
    month_diff.append(months_back + (12 * years_back))

polls["month_diff"] = pd.Series(month_diff)

In [9]:
# Create a weighting column for recency of polling data
polls["recency_weight"] = 1 / (polls["month_diff"] + 1)

## Weight for Pollster Rating

In [10]:
# Create a weighting column for quality of pollster: from FiveThirtyEight
polls["pollster_weight"] = 1 / (polls["Predictive Plus-Minus"] + 2.0)

In [11]:
# Create cumulative weighting column
polls["poll_weight"] = polls["pollster_weight"] * polls["recency_weight"]

In [12]:
# Create weighted polls columns
polls["weighted_votes_dem"] = polls["poll_weight"] * polls["votes_dem"]
polls["weighted_votes_rep"] = polls["poll_weight"] * polls["votes_rep"]

In [13]:
polls

,question_id,poll_id,stage,office_type,pollster,state,end_date,candidate_name_rep,candidate_party_rep,sample_size,...,votes_dem,total_decided,month,year,month_diff,recency_weight,pollster_weight,poll_weight,weighted_votes_dem,weighted_votes_rep
0,113994,62978,general,U.S. President,Emerson College,New Hampshire,2019-11-26,Donald Trump,REP,547,...,266,546,11,2019,0,1.000000,0.666667,0.666667,177.333333,186.666667
1,113999,62978,general,U.S. President,Emerson College,New Hampshire,2019-11-26,Donald Trump,REP,637,...,301,574,11,2019,0,1.000000,0.666667,0.666667,200.666667,182.000000
2,114109,62980,general,U.S. President,SurveyUSA,California,2019-11-22,Donald Trump,REP,2039,...,1141,1854,11,2019,0,1.000000,0.833333,0.833333,950.833333,594.166667
3,113252,62668,general,U.S. President,SurveyUSA,Georgia,2019-11-18,Donald Trump,REP,1303,...,599,1185,11,2019,0,1.000000,0.833333,0.833333,499.166667,488.333333
4,113346,62674,general,U.S. President,Marquette University Law School,Wisconsin,2019-11-17,Donald Trump,REP,801,...,344,728,11,2019,0,1.000000,1.000000,1.000000,344.000000,384.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,92980,57445,general,U.S. President,Emerson College,Iowa,2019-02-02,Donald Trump,REP,831,...,333,736,2,2019,9,0.100000,0.666667,0.066667,22.200000,26.866667
103,92929,57427,general,U.S. President,Glengariff Group Inc.,Michigan,2019-01-26,Donald Trump,REP,600,...,274,529,1,2019,10,0.090909,0.384615,0.034965,9.580420,8.916084
104,92872,57407,general,U.S. President,Praecones Analytica,New Hampshire,2019-01-21,Donald Trump,REP,593,...,317,562,1,2019,10,0.090909,0.344828,0.031348,9.937304,7.680251
105,92602,57275,general,U.S. President,Public Policy Polling,North Carolina,2019-01-07,Donald Trump,REP,750,...,345,690,1,2019,10,0.090909,0.454545,0.041322,14.256198,14.256198
